In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from gymnasium.wrappers import FlattenObservation
from stable_baselines3.common.env_checker import check_env
import numpy as np
import matplotlib.pyplot as plt
from Utils.TradingStation import TradingEnvironment

In [2]:
# ====================
# ENTRENAMIENTO
# ====================
render_global = False
# Crear el entorno de entrenamiento sin renderizar
base_env = TradingEnvironment(csv_path='US30.cash_data_m1_antiguo.csv',render_mode=None)
#base_env = gym.make("CartPole-v1")
check_env(base_env, warn=True, skip_render_check=True)

Duplicated index found. Removing duplicates...


In [ ]:
def make_env(rend_mode=None):
    # Define la ruta al archivo CSV con tus datos
    csv_path = 'US30.cash_data_m1_antiguo.csv'  # datos para simulación (velas de un minuto)
    # Crea la instancia del entorno
    env = TradingEnvironment(csv_path, render_mode=rend_mode, sl=30, tp=40, balance=10000, risk=240)
    # Envuelve el entorno para convertir las observaciones tipo diccionario en un vector plano
    # Monitor para registrar estadísticas y episodios
    env = Monitor(env, filename='logs/monitor.csv')
    return env

# Crea un DummyVecEnv a partir de la función make_env
env = DummyVecEnv([make_env])
# Normaliza las observaciones y recompensas
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10., norm_obs_keys=["distance_p","distance_fl"])

# Crea el modelo PPO utilizando una política MLP
model = PPO("MultiInputPolicy", env, verbose=1,device="cpu")

# Entrena el agente
total_timesteps = 3000000  # Ajusta según tus necesidades
model.learn(total_timesteps=total_timesteps)

Duplicated index found. Removing duplicates...
Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.3     |
|    ep_rew_mean     | 1.95     |
| time/              |          |
|    fps             | 904      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 29.1        |
|    ep_rew_mean          | 4.48        |
| time/                   |             |
|    fps                  | 733         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007776407 |
|    clip_fraction        | 0.045       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    exp

In [5]:
# Guarda el modelo entrenado y los stats de normalización
model.save("ppo_trading_model")
env.save("vec_normalize.pkl")

In [19]:
env_test = DummyVecEnv([make_env])
env_test = VecNormalize.load("vec_normalize.pkl", env_test)
model = PPO.load("ppo_trading_model.zip",device="cpu")
env_test.training = False
env_test.norm_reward = False

Duplicated index found. Removing duplicates...


In [ ]:
vars = [-15, 9500, 1, -450, 1]
obs_externa = {
    "distance": np.array([vars[0]], dtype=np.float32),
    "balance": np.array([vars[1]], dtype=np.float32),
    "trend": vars[2],  # Aunque trend es discreto, hay que incluirlo
    "daily_balance": np.array([vars[3]], dtype=np.float32),
    "fase": vars[4]
}
obs_norm = env_test.normalize_obs(obs_externa)
action_predicted,_states=model.predict(obs_norm, deterministic=True)
print(action_predicted)

0


In [ ]:
# Ejemplo de evaluación: ejecuta el agente en el entorno
obs = env_test.reset()
for _ in range(200):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, infos = env.step(action)
    env.render()

In [6]:
eliminar=base_env = TradingEnvironment(csv_path='US30.cash_data_m1_antiguo.csv',render_mode=None)

Duplicated index found. Removing duplicates...
